In [ ]:
// Importieren der benötigten Module
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FsODE, 0.0.1"
#r "nuget: FSharp.Stats"

// ... und öffnen der benöätigten Module
open FsODE
open Plotly.NET
open FSharp.Stats


// der Rechenalgorithmus der Programmbibliothek, die wir benutzen:
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

Installed Packages FSharp.Stats, 0.4.7 FsODE, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

## Modellierung von einfachen biologischen Systemen

### Inhalt

- Biologische Systeme
- Aktivator und Repressor
- Autoregulation
    - Negative Autoregulation
    - Postive Autoregulation

### Biologische Systeme 

Biologische Systeme können als Netzwerke aufgefasst werden, in welchen biologischen Einheiten miteinander verknüpft sind. Dabei hängen Größe, Art, und Anzahl der biologischen Einheiten vom gewählten System ab. Biologische Zellen sind dabei besonders interessanteste Systeme. Sie werden begrenzt von einer Membran die Stoff, Energie, und Informationsaustausch mit der Umgebung erlaubt, und enthalten Millionen von Molekülen die alle miteinander interagieren. Dabei sind die Interaktionen zumeist spezifischer Natur und formen selbst wiederum kleinere biologische Systeme, die die Funktion der ganzen Zelle sicherstellen. Beispiele für solche Systeme sind die Kontrolle des Zelltods, Signalwege die die Kommunikation mit anderen Zellen erlauben, die Nutzung von Metaboliten, und viele weitere. 
Die Grundeinheiten in solchen Systemen sind Gene, beziehungsweise die Proteine, welche von diesen kodiert werden. Daher ist es notwendig herauszufinden, wie hoch die Konzentration der einzelnen Proteine zu einem bestimmten Zeitpunkt ist und wie diese sich über die Zeit verändert. Dieses Verhalten kann allgemein durch ODEs beschrieben werden, wobei der Aufbau der Differenzialgleichung immer identisch ist und sich nur die Komplexität von dieser ändert, in Abhängigkeit der Anzahl an Faktoren, die auf die Produktion des Proteins Einfluss nehmen. Diese allgemeine Form ist dabei
$$ \frac {dProteinkonzentration} {dt} = \textrm{Produktion des Proteins} - \textrm{Verringerung der Proteinkonzentration}$$

Was ist dann die Proteinkonzentration eines Proteins X das von einem Gen x produziert wird? Wenn die Annahme getätigt wird, dass das Gen x nur das Protein X codiert und auch in sonst keiner Weise beeinflusst wird, kann die Produktion des Proteins X direkt mit einer Produktionsrate $\beta$ beschrieben werden. Die Verringerung der Proteinkonzentration von X ist auf der einen Seite abhängig vom aktiven Abbau in der Zelle, und auf der anderen von der Verteilung des Proteins in der Zelle während diese wächst und sich teilt. Der einfachheitshalber, können diese Faktoren allerdings fast immer als ein Prozess betrachtet werden. Die mathematisch Beschreibung dieses Prozesses ist dann eine Verringerung der Proteinkonzentration proportional zu einem Verringerungsfaktor $\gamma$. Die Differenzialgleichung ist somit

$$ \frac {dProteinkonzentration} {dt} = \beta - \gamma*Proteinkonzentration$$

Aufgabe:
Setze die Differenzialgleichung in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole).


In [ ]:
//Konstanten
let beta =    1. // Konstante für die Zunahme der Proteinkonzentration
let gamma =   5. // Konstante für die Verringerung der Proteinkonzentration

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->
        // Veränderung der Proteinmenge
        let Proteinkonznetration = P[0] 
        // Modelfunktion (hier die Gleichung einfügen)
        let Proteinkonznetration' = beta - gamma*Proteinkonznetration

        [| Proteinkonznetration' |]

//Startkonzentrationen
let P0 = [| 
    10.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

let proteins = Ps |> SolPoints.toPoints 1

//Graphische Darstellung der numerischen Lösung
proteins
|> Chart.Spline 
|> Chart.withTraceInfo("Protein")
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

verändert. Die Proteinkonzentration startet am Zeitpunkt 0 bei einem Wert von 10, $Proteinkonzentration (0) = 10$. Die Proteinkonzentration nähert sich mit der Zeit dem Wert 0.2. Dieser Wert entspricht dem Punkt an der Neubildung von Proteinen und Verringerung der Proteinanzahl im Gleichgewicht stehen. Dieser Wert wird im folgenden $Proteinkonzentration_{GGW}$ genannt. Das $Proteinkonzentration_{GGW}$ ist von den Werten von $\beta$ und $\gamma$ anhängig. Die Werte von $\beta$ und $\gamma$ die zu dem $Proteinkonzentration_{GGW} = 0.2$ geführt haben sind 1. und 5. (die Werte sind im oberen code block beim Absatz Konstanten zu finden).

Aufgabe: 
 Verändere den Wert von $\beta$ und versuche eine Gesetzmäßigkeit zu erkennen die beschreibt welchen Wert die Proteinkonzentration im GGW annimmt in Abhängigkeit von $\beta \textrm{ und } \gamma$. Wähle die korrekte Gleichung aus den vier nachfolgen aus, die diese Gesetzmäßigkeit beschreibt:


a) $Proteinkonzentration_{GGW} = \gamma  - 2*\beta$
 
b) $Proteinkonzentration_{GGW} = \frac {\beta * \gamma}  {\gamma^{2}}$ 

c) $Proteinkonzentration_{GGW} = \frac {\beta}  {\gamma}$ 

d) $Proteinkonzentration_{GGW} = \frac {\beta^{2}}  {\gamma^{2}}$ 

### Aktivator und Repressor 

Leider ist die die Biologie in fast allen fällen nicht so einfach. Stattdessen werden Gene in der Regel reguliert, um die Genexpression an die Umgebungsbedingungen anzupassen. Die einfachste Form der Regulierung ist mittels Repressoren oder Aktivatoren. Beides sind zumeist Proteine, die an spezifische Regionen auf der DNA binden können um die Promotoraktivität eines Gens zu regulieren, wobei die Stärke der Regulierung meist mit der Stärke der Bindung des Regulators zur DNA-Region einhergeht. <br>
Um den Einfluß eines Repressors/Aktivators auf die Produktion von einem Protein zu bestimmen, ist es zuerst nötig sich mit dem Prozess der Bindung des Repressors/Aktivators an die jeweilige Bindungsstelle, durch die die Proteinprodukion beeinflusst wird, zu beschäftigen. Innerhalb der Zelle bindet und löst sich ein Repressor von seiner Bindungsstelle in einem kontinuierlichen Prozess. Dies kann als eine Gleichgewichtsreaktion dargestellt werden
$$D + R\quad \underset{k_{-}}{\stackrel{k_{+}}{\rightleftharpoons}} \quad D_{gebunden}$$
wobei $D$ die Konzentration von Bindungsstellen ohne Repressor, $R$ die Repressorkonzentration, $D_{gebunden}$ die Konzentration von Bindungsstellen mit Repressor, $k_{+}$ die Geschwindigkeit des Bindens des Repressor an die Bindungstelle, und $k_{-}$ die Geschwindigkeit des Loslösens des Repressors von der Bindungstelle ist. $D$ und $D_{gebunden}$ zusammen ergeben $D_{total}$ was der Konzentration aller Bindungstellen entspricht ($D_{total} = D + D_{gebunden} $).<br>
Das Gen wird nur dann stark exprimiert wenn der Repressor nicht gebunden ist, was dem Verhältnis von nicht gebundenen Bindungstellen zu der Anzahl von allen Bindungsstellen entspricht, also $D/D_{total}$. Für eine Differenzialgleichung die beschreibt wie sich die Proteinkonzentration verhält wenn das Gen von einem Repressor reguliert wird ist es daher nötig die Produktionsrate mit dem Verhältnis $D/D_{total}$ zu multiplizieren. Sowohl $D$ als auch $D_{total}$ sind aber nur schwierig experimentell zu bestimmen. Die Repressorkonzentration und die Dissoziationskonstante, also die Konzentration  bei der die Produktion um die Hälfte reduziert ist, sind hingegen vergleichsweise einfach zu bestimmen. Daher ist es sinnvoll das Verhältnis $D/D_{total}$ als einen Zusammenhang von $R$ und der Dissoziationskonstante $K_{D}$ darzustellen. Dies ist möglich durch das Umformen der Gleichgewichtsgleichung die sich aus der Gleichgewichtsreaktion ergibt. Diese ist
$$k_{+}*D*R = k_{-}*D_{gebunden}$$
Einsetzten von $D_{total} = D + D_{gebunden} $ um $D_ {gebunden}$ zu ersetzen ergibt 
$$k_{+}*D*R = k_{-}*(D_{total}-D)$$
Seperieren von $D$ und $D_{total}$ von den anderen Faktoren führt zu
$$\frac {D} {(D_{total}-D)}= \frac {k_{-}}{k_{+}*R} $$
$$\frac {D/D} {(D_{total}/D-D/D)}= \frac {k_{-}}{k_{+}*R} $$
$$\frac {1} {(D_{total}/D-1)}= \frac {k_{-}}{k_{+}*R} $$
$$\frac {D_{total}}{D}-1= \frac {k_{+}*R}{k_{-}} $$
$$\frac {D_{total}}{D}= \frac {k_{+}*R}{k_{-}}+1 $$
das Verhältnis von $\frac {k_{-}}{k_{+}}$ kann ersetzt werden durch die Dissoziationskonstante $K_{D}$. Daher folgt
$$\frac {D_{total}}{D}= \frac {R}{K_{D}} +1 $$
$$\frac {D}{D_{total}}= \frac {1}{1+\frac {R}{K_{D}}} $$

Der Term $\frac {1}{1+\frac {R}{K_{D}}}$ beschreibt somit den Term durch den die Expressionsrate eines Gens gedämpft wird, wenn es von einem Repressor reguliert wird. Das heißt wenn das Gen x, welches das Protein X produziert, durch den Inhibitor R reguliert wird, kann dies mit folgender Differenzialgleichung beschrieben werden
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ \frac {R}{K_{D}}}  - \gamma*Proteinkonzentration$$





Aufgabe:<br>
Setze die Differenzialgleichungen für ein Protein dessen Produktion von einem Repressor inhibiert wird in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).


In [ ]:
//Konstanten
let alpha =    0.5  // Konstante für die Zunahme der Proteinkonzentration (nicht reguliert)
let beta =    50.0  // Konstante für die Zunahme der Proteinkonzentration (reguliert)
let gamma =   20.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =     20.0  // Repressorkonzentration bei der die Proteinproduktion um die Hälfte reduziert ist
let R =       10.0  // Repressorkonzentration


// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let x = P[0] // Änderung in der Proteinmenge (unreguliert)
        let Proteinkonznetration = P[1] // Änderung in der Proteinmenge (reguliert durch Repressor)
        //Änderung der Proteinkonzentration ohne einen Repressor (nicht verändern bitte)
        let x' = beta - (gamma*x)
        // Änderung des Proteinkonzentration unter Einfluß von einem Repressor (hier die Gleichung einsetzen)
        let Proteinkonznetration' = beta * 1./(1.+(R/K_d)) - (gamma*Proteinkonznetration)

        [| x'; Proteinkonznetration' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0  
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        1., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzhal der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = Ps |> SolPoints.toPoints 2 
    proteinsregulated|> Chart.Spline 
    |> Chart.withTraceInfo("Reguliert");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Aufgabe: <br>
Erklären sie wie sich der Repressor auf die Höhe des Gleichgewichtszustandes der Proteinkonzentration, $Proteinkonzentration_{GGW}$, und auf die Zeit die benötigt wird diesen zu erreichen auswirkt. Verändern sie dafür die Werte von "R" falls nötig. Beachten sie das die eingesetzten Werte reale Zahlen sein müssen und nicht ganze Zahlen.


Aufgabe: <br>
In der Realität wird ein Gen schwach exprimiert, auch wenn die Expression des Gens durch einen Repressor gehemmt wird. Ergänze die Differenzialgleichungen für ein Protein dessen Produktion von einem Repressor inhibiert wird um einen Term $\alpha$ der diese Grundgenexpression wiederspielt. Beachte dabei das $\alpha$ nicht vom Repressor beeinflusst wird (setze $\alpha$ als "alpha", ohne "", in die Gleichung ein).<br>
Wie wirkt sich $\alpha$ auf den Gleichgewichtszustand aus?


Aufgabe:<br>
Stelle die Differenzialgleichung für die Proteinproduktion auf, die von einem Aktivator reguliert wird. Beachte dabei das ein Aktivator gegensätzlich zu einem Repressor wirkt und daher der Faktor mit dem $beta$ multipliziert werden muss $\frac {D_{gebunden}}{D_{total}}$ ist. Dies bedeutet das sie folgende Gleichung in diese Form umwandeln müssen
$$k_{+}*(D_{gebunden}-D_{total})*R = k_{-}*D_{gebunden}$$
und anschließend in die Differenzialgleichung für die Proteinproduktion einsetzen müssen
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {D_{gebunden}}{D_{total}}  - \gamma*Proteinkonzentration$$ 

Falls sie Probleme mit dem Umformen haben, schauen sie sich noch einmal die Umformung für den Repressor an.

Setze die Differenzialgleichungen für ein Protein das von einem Aktivator und die Differenzialgleichungen für ein Protein das von einem Repressor reguliert wird in das folgende Model ein (beachten sie dabei, das sie die Proteinkonzentration für die Gleichung die von einem Repressor beeinflusst wird "Proteinkonzentration2, ohne "", nennen und die Proteinkonzentration für die Gleichung die von einem Aktivator beeinflusst wird "Proteinkonzentration3, ohne "", nennen. Bechte ebenfalls das  $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird)


In [ ]:


//Konstanten
let beta =    10.  // Konstante für die Zunahme der Proteinkonzentration
let gamma =   5.   // Konstante für die Verringerung der Proteinkonzentration
let K_d =     2.   // Repressorkonzentration bei der die Proteinproduktion um die Hälfte reduziert ist
let R =       10.  // Repressorkonzentration
let A =       10.  // Aktivatorexpression 

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonzentration1 = P[0] // Änderung in der Proteinmenge (unreguliert)
        let Proteinkonzentration2 = P[1] // Änderung in der Proteinmenge (reguliert durch Repressor)
        let Proteinkonzentration3 = P[2] // Änderung in der Proteinmenge (reguliert durch Aktivator)
        //Änderung der Proteinkonzentration ohne einen Repressor
        let Proteinkonzentration1' = beta - (gamma*Proteinkonzentration1)
        // Änderung des Proteinkonzentration unter Einfluß von einem Repressor (hier die Gleichung für die Proteinproduktion reguliert durch einen Repressor einfügen)
        let Proteinkonzentration2' = beta * 1./(1.+(R/K_d)) - (gamma*Proteinkonzentration2)
        // Änderung des Proteinkonzentration unter Einfluß von einem Aktivator (hier die Gleichung für die Proteinproduktion reguliert durch einen Aktivator einfügen)
        let Proteinkonzentration3' = beta * 1./(1.+(K_d/A)) - (gamma*Proteinkonzentration3)

        [| Proteinkonzentration1'; Proteinkonzentration2';Proteinkonzentration3' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0
    0.0  
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulatedbyrep = Ps |> SolPoints.toPoints 2 
    proteinsregulatedbyrep|> Chart.Spline 
    |> Chart.withTraceInfo("Repressor");
    let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 3 
    proteinsregulatedbyakt|> Chart.Spline 
    |> Chart.withTraceInfo("Aktivator");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen:

Expertenaufgabe <br>
Aktivator und Repressor führen beide zu einer Verringerung der maximalen Genexpression. Wenn allerdings beide zum selben Ergebnis führen, warum gibt es dann beides? Erklären sie, warum mache Gene eher von einem Aktivator reguliert werden sollten als von einem Repressor und umgekehrt.

### Autoregulation

#### Negative Autoregulation

Neben der Fähigkeit von Repressoren die Produktion von bestimmten Proteinen zu regulieren, sind viele auch dazu in der Lage sich selbst zu regulieren. Ein Beispiel dafür ist der Repressor LacI, der nicht nur die Expression des Lac Operons begrenzt sondern auch seine eigene. Dies wird negative Autoregulation genannt. Die Differenzialgleichung, die diesen Prozess beschreibt ist analog zu die von der Proteinkonzentration die von einem Repressor reguliert wird, nur das die Repressorkonzentration ersetzt wird mit der Proteinkonzentration, also
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ \frac {Proteinkonzentration}{K_{D}}}  - \gamma*Proteinkonzentration$$

Aber warum sollte ein Protein seine eigene Produktion durch Autoregulation dämpfen? Wäre es nicht deutlich einfach die maximale Reproduktionsrate $\beta$ zu verringern, da eine Verringerung von $\beta$ (oder eine Zunahme von $\gamma$) zu einem geringerem $Proteinkonezntation_{GGW}$ führt (wie bereits bei Frage 2 bearbeitet)? Um diese Frage zu beantworten ist es nötig sich die Zeit anzusehen die benötigt wird um das $Proteinkonezntation_{GGW}$ zu erreichen. Das folgende Model zeigt wie sich die Proteinkonzentration, für ein Protein das ohne weitere Einflüße expremiert wird (also ohne Eigenregulation, oder Repressor/ Aktivator), verhält für verschiedene Werte von $\beta$ und $\gamma$. Beachte dabei das wenn $\beta$ varriert wird (Graph 1) gamma konstant bleibt, und wenn $\gamma$ varriert wird (Graph 2)  $\beta$ konstant bleibt.




In [ ]:
//Bitte nichts ändern!

//Konstanten
let beta1 =   0.5// Konstante für die Zunahme der Proteinkonzentration
let beta2 =   1. // Konstante für die Zunahme der Proteinkonzentration
let beta3 =   2. // Konstante für die Zunahme der Proteinkonzentration
let beta4 =   4. // Konstante für die Zunahme der Proteinkonzentration
let gamma1 =  2. // Konstante für die Verringerung der Proteinkonzentration
let gamma2 =  4. // Konstante für die Verringerung der Proteinkonzentration
let gamma3 =  8. // Konstante für die Verringerung der Proteinkonzentration
let gamma4 =  16.// Konstante für die Verringerung der Proteinkonzentration


// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->
        // Veränderung der Proteinmenge
        let Proteinkonznetration1 = P[0] // Änderung in der Proteinmenge
        let Proteinkonznetration2 = P[1] // Änderung in der Proteinmenge
        let Proteinkonznetration3 = P[2] // Änderung in der Proteinmenge
        let Proteinkonznetration4 = P[3] // Änderung in der Proteinmenge

        let Proteinkonznetration5 = P[4] // Änderung in der Proteinmenge
        let Proteinkonznetration6 = P[5] // Änderung in der Proteinmenge
        let Proteinkonznetration7 = P[6] // Änderung in der Proteinmenge
        let Proteinkonznetration8 = P[7] // Änderung in der Proteinmenge
        // Modelfunktion
        let Proteinkonznetration1' = beta1 - gamma2*Proteinkonznetration1 //DGL für ein unreguliertes Protein
        let Proteinkonznetration2' = beta2 - gamma2*Proteinkonznetration2 //DGL für ein unreguliertes Protein
        let Proteinkonznetration3' = beta3 - gamma2*Proteinkonznetration3 //DGL für ein unreguliertes Protein
        let Proteinkonznetration4' = beta4 - gamma2*Proteinkonznetration4 //DGL für ein unreguliertes Protein

        let Proteinkonznetration5' = beta3 - gamma1*Proteinkonznetration5 //DGL für ein unreguliertes Protein
        let Proteinkonznetration6' = beta3 - gamma2*Proteinkonznetration6 //DGL für ein unreguliertes Protein
        let Proteinkonznetration7' = beta3 - gamma3*Proteinkonznetration7 //DGL für ein unreguliertes Protein
        let Proteinkonznetration8' = beta3 - gamma4*Proteinkonznetration8 //DGL für ein unreguliertes Protein

        [|Proteinkonznetration1';Proteinkonznetration2';Proteinkonznetration3';Proteinkonznetration4';Proteinkonznetration5';Proteinkonznetration6';Proteinkonznetration7';Proteinkonznetration8' |]

//Startkonzentrationen
let P0 = [| 
    0.0 
    0.0 
    0.0 
    0.0 
    0.0
    0.0
    0.0
    0.0
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 15 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

// Generierung der Punkte für das Kennzeichnen der Antwortszeit
let t0 = log(2.) / gamma2
let x1 = beta1 / gamma2 * (0.5)
let x2 = beta2 / gamma2 * (0.5)
let x3 = beta3 / gamma2 * (0.5)
let x4 = beta4 / gamma2 * (0.5)
let t1 = log(2.) / gamma1
let t2 = log(2.) / gamma2
let t3 = log(2.) / gamma3
let t4 = log(2.) / gamma4
let x5 = beta3 / gamma1 * (0.5)
let x6 = beta3 / gamma2 * (0.5)
let x7 = beta3 / gamma3 * (0.5)
let x8 = beta3 / gamma4 * (0.5)

//Graphische Darstellung der numerischen Lösung
[
    [
        let proteins = Ps |> SolPoints.toPoints 1 
        proteins |> Chart.Spline 
        |> Chart.withTraceInfo("beta = 0.5");
        let proteinsregulatedbyrep = Ps |> SolPoints.toPoints 2 
        proteinsregulatedbyrep|> Chart.Spline 
        |> Chart.withTraceInfo("beta = 1.0");
        let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 3 
        proteinsregulatedbyakt|> Chart.Spline 
        |> Chart.withTraceInfo("beta = 2.0");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword ForestGreen)
        let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 4 
        proteinsregulatedbyakt|> Chart.Spline 
        |> Chart.withTraceInfo("beta = 4.0");
        [t0,x1]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Blue)
        |> Chart.withTraceInfo("beta = 0.5");
        [t0,x2]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Red)
        |> Chart.withTraceInfo("beta = 1.0");
        [t0,x3]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Green)
        |> Chart.withTraceInfo("beta = 2.0");
        [t0,x4]
        |> Chart.Point 
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Purple)
        |> Chart.withTraceInfo("beta = 4.0");
    ]
    |> Chart.combine
    |> Chart.withTitle @"Veränderung von beta"
    |> Chart.withXAxisStyle("Zeit")
    |> Chart.withYAxisStyle("Proteinkonzentration")

    [
        let proteins = Ps |> SolPoints.toPoints 5 
        proteins |> Chart.Spline 
        |> Chart.withTraceInfo("gamma = 2.");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword DarkViolet)
        let proteinsregulatedbyrep = Ps |> SolPoints.toPoints 6 
        proteinsregulatedbyrep|> Chart.Spline 
        |> Chart.withTraceInfo("gamma = 4.");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword ForestGreen)
        let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 7 
        proteinsregulatedbyakt|> Chart.Spline 
        |> Chart.withTraceInfo("gamma = 8.");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Red)
        let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 8 
        proteinsregulatedbyakt|> Chart.Spline 
        |> Chart.withTraceInfo("gamma = 16.");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Blue)
        [t1,x5]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword DarkViolet)
        |> Chart.withTraceInfo("gamma = 2.");
        [t2,x6]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Green)
        |> Chart.withTraceInfo("gamma = 4.0");
        [t3,x7]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Red)
        |> Chart.withTraceInfo("gamma = 8.0");
        [t4,x8]
        |> Chart.Point 
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Blue)
        |> Chart.withTraceInfo("gamma = 16.0");
    ]
    |> Chart.combine
    |> Chart.withTitle @"Veränderung von beta                                       Veränderung von gamma"
    |> Chart.withXAxisStyle("Zeit")
    |> Chart.withYAxisStyle("Proteinkonzentration")

]
|> Chart.Grid (1,2)
|> Chart.withSize(1000,500)

<!-- Plotly chart will be drawn inside this DIV -->

Die Graphen zeigen das sowohl die Abnhame von $\beta$ als auch die Zunahme von $\gamma$ zu einer Verringerung des $Proteinkonzentration_{GGW}$ führt. Allerdings unterscheidet sich die Form der Graphen erheblich. 
Acht Punkte sind makiert die diesen Fakt unterstreichen. Diese repräsentieren die Antwortszeit welche beschreibt wie lange es dauert bis der Wert der $Proteinkonzentration (t)$ gleich dem Wert $\frac {1}{2}*Proteinkonzentration_{GGW}$ ist. 
Die Betrachtung der Antwortszeit zeigt, dass eine Veränderung von $\beta$ nicht zu einer Verkürzung der Antwortszeit führt, während eine Veränderung von $\gamma$ zu einer Verringerung der Antwortszeit führt. Die Formel für die Berechnung der Antwortszeit ist
$$ T_{\frac {1}{2}} = \frac  {\log (2)}{\gamma}$$
wobei $T_{\frac {1}{2}}$ die Antwortszeit repräsentiert. Dies bedeutet wenn ein Protein möglichst schnell produziert werden soll, sollte der Wert für $\gamma$ möglichst groß sein (Anmerkung: Die gezeigte Formel gilt nur für unregulierte Proteine).

Aufgabe: <br>
Überlege was ein hoher Wert für $\gamma$ im Bezug auf die Proteinstabilität bedeutet (zur Erinnerung $\gamma$ beschreibt sowohl den aktiven Abbau, als auch die Verteilung in der Zelle beim Wachstum).
Tipp: Falls sie Probleme mit dieser Aufgabe haben, überlegen sie sich welche Faktoren einen schnellen beziehungsweise langsamen Abbau von einem Protein begünstigen. 



Betrachten sie nun den Graphen für ein Protein das sich selbst negativ Autoreguliert. <br>
Aufgabe: <br>
Setzen sie die Gleichung für negative Autoregulation in die nachfolgende Simulation ein (Bitte beachten sie, dass sie die Proteinkonznetration als "Proteinkonznetration2" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).

In [ ]:
//Konstanten
let beta  =  2.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0 // Konstante für die Verringerung der Proteinkonzentration
let K_d =    1.0 // Dissoziationskonstante

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonznetration1 = P[0] // Änderung in der Proteinmenge (unreguliert)
        let Proteinkonznetration2 = P[1] // Änderung in der Proteinmenge (reguliert durch negative Autoregulation)
        //Änderung der Proteinkonzentration ohne negative Autoregulation
        let Proteinkonznetration1' = beta - (gamma*Proteinkonznetration1)
        // Änderung des Proteinkonzentration bei negativer Autoregulation (hier die Gleichung einsetzen)
        let Proteinkonznetration2' = beta * 1./(1.+(Proteinkonznetration2/K_d)) - (gamma*Proteinkonznetration2)

        [| Proteinkonznetration1'; Proteinkonznetration2' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0  
|]

// Generierung der Punkte für das Kennzeichnen der Antwortszeit
let t0 = log(2.) / gamma
let t1 = K_d/(2. * beta) + 0.135
let x0 = beta / gamma * (0.5) 
let x1 = K_d * 0.5 

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 12 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = Ps |> SolPoints.toPoints 2 
    proteinsregulated|> Chart.Spline 
    |> Chart.withTraceInfo("negative AR");
    [t0,x0]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Blue)
        |> Chart.withTraceInfo("Unreguliert");
    [t1,x1]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Red)
        |> Chart.withTraceInfo("negative AR");    
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Aufgabe: <br>
Welchen Effekt hat die negative Autoregulation im Bezug auf die Antwortzeit 

Aufgabe: <br>
Warum ist negative Autoregulation ein sinnvollerer Mechanismus für eine verkürzung der Antwortszeit für die meisten Proteine im Vergleich zur Erhöhung von $\gamma$?


Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen:


Aufgabe: <br>
Überlegen sie sich wie lange die Antwortszeit von einem Protein ist, mit folgenden Bedingungen:
1. Es gibt eine gewisse Anfangskonzentration $P_{Start}$
2. Die Produktionsrate $\beta$ ist null
3. Der Teil $\gamma_{Abbau}$ von $\gamma$ ist null. 

Das bedeutet die initale Proteinkonzentration verändert sich nur durch das Verteilen des Proteins in der Zelle und durch Zellteilung. Die Antwortszeit soll sich auf das Volumen von einer Zelle beziehen.
Überlegen sie weiterhin, ob diese Bedingungen häufig auftreten insbesondere mit Blick auf die Stabilität der meisten Proteine in der Zelle.

#### Postive Autoregulation

Die Betrachtung der negativen Autoregulation zeigte das sie dazu genutzt werden kann die Antwortgeschwindigkeit der Proteinkonznetration zu erhöhen. Beim Vergleich von Repressor and Aktivator hatten beide einen identischen Effekt, beide verringerten die $Proteinkonzentration_{GGW}$ und hatten keinen Einfluß auf die Antwortsgeschwindigkeit. Daher ist die logische Folgerung, dass sich postive Autoregulation genauso wie die negative Autoregulation verhält. Diese Vermutung soll im nachfolgendem Abschnitt untersucht werden. Eine durch positive Autoregulation regulierte Proteinproduktion wird durch folgende Differenzialgleichung dargestellt

$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ \frac {Proteinkonzentration}{K_{D}}}  - \gamma*Proteinkonzentration$$

Allerdings ist es an dieser Stelle relevant eine andere Darstellung einzuführen. Die meisten Antworten in der Genregulkation und bei Protein-Protein-Interaktionen haben ein **ultrasensitives** Verhalten. Das heißt das ein Prozess nur dann stattfindet wenn ein bestimmter Schwellenwert überschritten wird, während es davor keinen Effekt gibt. Oft ist dieses ultrasensitive Verhalten bedingt durch **Kooperativität**. Ein Beispiel dafür wäre Hemoglobin. Hemoglobin bindet bis zu vier Moleküle Sauerstoff. Wenn kein Molekül Sauerstoff gebunden ist, befindet sich das Hemoglobin in dem sogenannten T-State. In diesem ist die Affinität von Hemoglobin für Sauerstoff sehr gering und eine Bindung von Sauerstoff ist eher unwahrscheinlich. Fall aber ein Sauerstoffmolekül dennoch an das Hemoglobin bindet kommt es zu einer Konformationsänderung. Durch diese Änderung wird die Affinität von Hemoglobin für Sauerstoff deutlich erhöht und es ist wahrscheinlich das drei weitere Sauerstoffmolekükle an das Hemoglobin binden. Sobald alle vier Sauerstoffmoleküle an das Hemoglobin gebunden haben wird von dem sogenannten R-State gesprochen.<br>
Das Verhalten kann daher als ultrasensitiv beschrieben werden, weil es nur bei hoch genugen Sauerstoffkonzentration zu der Bildung des R-states kommt. Bei niedrigen Konzentration bnindet zwar ab und zu ein Molekül Sauerstoff an das Hemoglobin, aber weitere Sauerstoffmoleküle sind zu weit entfernt um die jetzt höhere Affinität auszunutzen, weshalb das gebunde Sauerstoff nach einiger Zeit das Hemoglobin wieder verlässt.<br>
Die beschreibung von solchen Ultrasensitiven Prozessen kann mit der sogenannten Hillfunktion beschrieben werden. Das anwenden von dieser auf die bisher besprochenen negative Autoregulation kann generisch mit der folgenen Differenzialgleichung beschrieben werden 
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ (\frac {Proteinkonzentration}{K_{D}})^{n}}  - \gamma*Proteinkonzentration$$

die Gleichung für eine ultrasentives Verhalten von einer Proteinproduktion die durch einen Repressor beeinflusst wird ist identisch, nur müssen sie die Proteinkonzentration im Term, der $\beta$ beeienflusst, durch die Repressorkonezntration ersetzten. Die DIfferenzialgleichung für postive Autoregulation ist 
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {Proteinkonzentration^{n}}{{(K_{D})^{n}} + Proteinkonzentration^{n}} - \gamma*Proteinkonzentration$$
auch hier führt das ersetzten der Protienkonzentration, im $\beta$ beeinflussenden Term, mit der Aktivatorkonezentration zu der Gleichung für eine Proteinproduktion die von diesem beieinflusst wird. 
Der Paramerter $n$ ist der Hillkoeffizent und beschreibt wie stark das ultrasensitve Verhalten ist. Ein Wert von 1 beschreibt die bisher besprochenen Verhaltensweisen, während ein Wert von 2 oder höher das ulatrsensitive Verhalten beschreibt. 

Aufgabe:

Setzen sie die Gleichung für postive Autoregulation in die nachfolgende Simulation ein, dabei sollte die postive Autoregulation durch eine Hill Funktion beschrieben werden (Bitte beachten sie, dass sie die Proteinkonznetration als "Proteinkonznetration2" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).

In [ ]:
//Konstanten
let beta  =  10.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =    4.0  // Dissoziationskonstante
let n =      1.0  // Hill Koeffizent 

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonznetration1 = P[0] // Änderung in der Proteinmenge
        let Proteinkonznetration2 = P[1] // Änderung in der Proteinmenge
        //Änderung der Proteinkonzentration ohne positive Autoregulation
        let Proteinkonznetration1' = beta - (gamma*Proteinkonznetration1)
        // Änderung des Proteinkonzentration bei positiver Autoregulation (hier die Gleichung einsetzen)
        let Proteinkonznetration2' = beta * (Proteinkonznetration2**n)/((Proteinkonznetration2**n) + (K_d**n)) - (gamma*Proteinkonznetration2)

        [| Proteinkonznetration1'; Proteinkonznetration2' |]

//Startkonzentrationen
let P0 = [| 
    //Startkonzentration für das unregulierte Protein (muss nicht verändert werden)
    0.0 
    //Startkonzentration für das durch postive Autoreguliation regulierte Protein (hier bitte die Startwerte ändern)
    0.1  
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = Ps |> SolPoints.toPoints 2 
    proteinsregulated|> Chart.Spline 
    |> Chart.withTraceInfo("negative AR");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Die Konstanten für den gezeigten Graphen sind:
$$\beta = 10$$
$$\gamma= 1$$
$$n = 1$$
$$K_{D} = 1$$
$$Proteinkonzentration (0) = 0.1$$
Aufgabe: <br>
Welchen Effekt hat postive Autoregulation bei einem Wert von $n=1$ auf die Antwortszeit?

Aufgabe: <br>
Verändere den Wert von $n$ auf 2.
Was beobachten sie?<br>
Erhöhen sie jetzt Schrittweise den Wert von der $Proteinkonzentration(0)$, nutzen sie dabei die folgenden Werte:
$$0.5$$
$$1.0$$
$$2.0$$
$$3.0$$
$$4.0$$

Was können sie im Bezug auf das $Proteinkonzentration_{GGW}$ sagen?

Aufgabe: <br>

Wenn sie die jeweiligen $Proteinkonzentration_{GGW}$ werten müssten mit den folgenden zwei Optionen:
1. stabil; das heißt die $Proteinkonzentration(0)$ kann größer oder kleiner als $Proteinkonzentration_{GGW}$ sein, die $Proteinkonzentration(t)$ geht dennoch gegen $Proteinkonzentration_{GGW}$
2. instabil; das heißt $Proteinkonzentration(t)$ geht nur dann gegen $Proteinkonzentration_{GGW}$, wenn $Proteinkonzentration(0) = Proteinkonzentration_{GGW}$

welchen Option würde dann die jeweiligen $Proteinkonzentration_{GGW}$ erhalten?

Ein System das zwei stabile Fixpunkte hat wird auch istabiles System genannt. **Bistabilität** ist ein der Schlüsseleigenschaften der meisten biologischen Systeme, da das System auch dann an einem der beiden Fixpunkte verweilt, auch wenn es leichte Störungen oder leicht veränderte äußere Einflüße gibt. Das bedeutet ein biologisches System das sich um den Schutz gegen stark gestiegene Temperaturen kümmert und die dafür beötigten Proteine produizert wird nicht aktiv wenn sich die Temperatur nur leicht ändert, sondern erst wenn es zu einer starken Abweichung von den normalern Bedingungen kommt. Dies ermöglicht einen effizenteren Einsatz von Resourcen, es kommt nur dann zu einer Proteinproduktion wenn diese benötigt werden, und damit auch eine bessere Überlebenswahrscheinlichkeit. <br>
Bistabilität ist allerdings nur eine spezialform der **Multistabilität**. Multistabilität bedeutet das ein System mehere Fixpunkte hat zu denen es wandern kann, anhängig von den äußeren Einflüßen. 


Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen.

Expertenaufgabe:

Teilen sie Gleichung welche die Proteinproduktion in Abhängigkeiten von ultrasenistver postiver Auporegulation beschreibt in zwei Gleichungen. Die erste soll die Produktion des Proteins beschreiben, die zweite die Abnahame, also
$$ \frac {dProteinkonzentration} {dt} = \textrm{Produktion des Proteins} $$
$$ \frac {dProteinkonzentration} {dt} = \textrm{Verringerung der Proteinkonzentration}$$

Fügen sie die beiden Gleichungen in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{d}$ als K_d eingesetzt wird). 

In [ ]:
//Konstanten
let beta  =  10.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =    4.0  // Dissoziationskonstante
let n =      2.0  // Hill Koeffizent 

//Graphische Darstellung der Gleichungen
[
    // Fügen sie die Gleichung für den Proteinabbau ein
    Chart.Line [ for x in 0.0 .. 0.2 .. 15.0 -> (x, (gamma*x)) ]
    |> Chart.withTraceInfo("Proteinabbau");
    //Fügen sie die Gleichung für die Proteinproduktion ein
    Chart.Line [ for x in 0.0 .. 0.2 .. 15.0 -> (x, beta * (x**n)/((x**n) + (K_d**n))) ]
    |> Chart.withTraceInfo("Proteinproduktion");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Varrieren sie die Werte für die Abbaukonstante $\gamma$. Bestimmen sie bei welchen Werten von $\gamma$ es zur Bistabilität kommt und bei welchen nicht. 

Take Home Messages
